# Refresh Data Base

## Description

...

The following cell will completly wipe the SQLite DB.

In [ ]:
import requests, sqlite3

def ParseCntrFile(fileData):
    r = requests.get("http://priv.carpenoctem.co/cntr/data/" + fileData["captureFileName"])

    #Get Players
    players = {}
    for line in r.text.splitlines():
        for event in line.split(';'):
            if event.startswith('U'):
                part = event.split(',')
                if part[9] == '1':
                    pId = part[1]
                    pName = part[3]
                    pRole = part[2]
                    pSquad = part[7]
                    if pName not in players:
                        players[pId] = {"Name": pName, "Role": pRole, "Squad": pSquad, "Deaths": 0, "Kills": 0}
                        #print(players[pId]["Name"])
    
    #Merge Duplicates
    delList = list()
    for key,val in players.items():
        count = 0
        for k,v in players.items():
            if val["Name"] == v["Name"]:
                count += 1
            if count > 1:
                players[key]["Deaths"] = players[k]["Deaths"]
                players[key]["Kills"] = players[k]["Kills"]
                delList.append(k)
    for key in delList:
        players.pop(key, None)
    #print(players)
                
    
    #Get Deaths
    deaths = {}
    kills = {}
    for line in r.text.splitlines():
        for event in line.split(';'):
            if event.startswith('K'):
                part = event.split(',')

                #Player Deaths
                if part[1] in players:
                    if part[1] not in deaths:
                        deaths[part[1]] = 1
                    else:
                        deaths[part[1]] += 1
                    #print(part[1])

                #Player Kills        
                if len(event.split(',')) >= 3 and event.split(',')[2] in players:
                    if part[2] in players:
                        if part[2] not in kills:
                            kills[part[2]] = 1
                        else:
                            kills[part[2]] += 1

    #Merge Dictonaries
    for pId, val in deaths.items():
        players[pId]["Deaths"] = val

    for pId, val in kills.items():
        players[pId]["Kills"] = val

    #Write to DB
    WriteToDB(players, fileData)

In [ ]:
import sqlite3

def WriteToDB(pDic, jData):
    conn = sqlite3.connect('stats.sqlite')
    c = conn.cursor()
    tDeaths = 0
    
    #Players
    for key,val in pDic.items():
        tDeaths += val["Deaths"]
        c.execute('SELECT 1 FROM Players WHERE Player=? LIMIT 1', (val["Name"],))
        exist = c.fetchone() is not None
        if exist == 0:
            c.execute("INSERT into Players values (?,?,?)", (val["Name"],1,val["Deaths"],))
        else:
            c.execute("UPDATE Players SET OperationCount = OperationCount + 1, DeathCount = DeathCount + ? WHERE Player = ?", (val["Deaths"], val["Name"]))
    conn.commit()
    
    #Operations
    for key,val in pDic.items():
        c.execute("INSERT into Operations values (?,?,?,?,?)", (val["Name"],jData["missionName"],jData["date"],jData["duration"],tDeaths,))
    conn.commit()
    
    #Roles
    for key,val in pDic.items():
        c.execute('SELECT 1 FROM Roles WHERE Player=? AND RoleName=? LIMIT 1', (val["Name"],val["Role"],))
        exist = c.fetchone() is not None
        if exist == 0:
            c.execute("INSERT into Roles values (?,?,?)", (val["Name"],val["Role"],1,))
        else:
            c.execute("UPDATE Roles SET RoleCount = RoleCount + 1 WHERE Player=? AND RoleName=?", (val["Name"],val["Role"],))
            
    conn.commit()
    conn.close()

In [ ]:
import requests, sqlite3

r = requests.get("http://priv.carpenoctem.co/cntr/data/index.json")

for data in r.json():
    if data["missionName"] != "Mess Around" and data["missionName"].find("Training",0) == -1 and data["duration"] > 1800:
        #Check DB for mission
        conn = sqlite3.connect('stats.sqlite')
        c = conn.cursor()
        c.execute('SELECT 1 FROM Operations WHERE OperationName=? AND OperationDate=? LIMIT 1', (data["missionName"],data["date"]))
        exist = c.fetchone() is not None
        conn.close()
        
        if exist == 0:
            print("Adding " + data["missionName"] + " to the database...")
            ParseCntrFile(data)